In [1]:
from tqdm import trange
import torch
import torch.nn as nn
from transformers import AdamW
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from transformers import GPT2Tokenizer

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
special_tokens = ["_start_", "_delimiter_", "_classify_"]
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', do_lower_case=True)
tokenizer.add_tokens(special_tokens)
special_tokens_ids = tokenizer.convert_tokens_to_ids(special_tokens)
model = torch.load("../models/Gpt2_Classifier_Large.pt")
model.resize_token_embeddings(len(tokenizer))

/home/test/anaconda3/envs/bots/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/test/anaconda3/envs/bots/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/test/anaconda3/envs/bots/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the origin

Embedding(50260, 768)

In [5]:
model = model.cuda()

We need to create our training dataset

In [6]:
directory = "../data/bot_detection/"

In [7]:
train = pd.read_csv(directory + "train.csv", header=None)

train = pd.DataFrame({
    'id':range(len(train)),
    'label':train[0],
    'mark':['a']*train.shape[0],
    'text': train[1].replace(r'\n', ' ', regex=True)
})

train.columns = ["index", "label", "mark", "tweet"]

In [8]:
test = pd.read_csv(directory + "test.csv", header=None)

test = pd.DataFrame({
    'id':range(len(test)),
    'label':test[0],
    'mark':['a']*test.shape[0],
    'text': test[1].replace(r'\n', ' ', regex=True)
})

test.columns =  ["index", "label", "mark", "tweet"]

## Preprocessing

In [9]:
train_sentences = train.tweet.values

In [10]:
test_sentences = test.tweet.values

We might only want those tweets with one hashtag, url or user

In [11]:
url_positions = []
hashtag_positions = []
user_positions = []
for train_sentence in train_sentences:
    train_sentence_split = train_sentence.split()
    urls = [i for i in train_sentence_split if (i.startswith('https:') or i.startswith('http:'))]
    if(len(urls)==1 and urls[0]==train_sentence_split[-1]): # If contains only a token, and this token is at the end of the tweet
        start = train_sentence.find(urls[0])
        end = start+len(urls[0])
        url_positions.append([start,end])
    else:
        url_positions.append([-1,-1])

    hashtags = [i for i in train_sentence_split if i.startswith('#')]
    if(len(hashtags)==1 and hashtags[0]==train_sentence_split[-1]):
        start = train_sentence.find(hashtags[0])
        end = start+len(hashtags[0])
        train_sentences[0][start:end]
        hashtag_positions.append([start,end])
    else:
        hashtag_positions.append([-1,-1])

    users = [i for i in train_sentence_split if i.startswith('@')]
    if(len(users)==1 and users[0]==train_sentence_split[-1]):
        start = train_sentence.find(users[0])
        end = start+len(users[0])
        train_sentences[0][start:end]
        user_positions.append([start,end])
    else:
        user_positions.append([-1,-1])

In [12]:
url_mask = np.all(np.array(url_positions)!=np.array([-1,-1]),axis=1)
np.array(url_positions)[url_mask]
train_sentences[url_mask]

array(['when you wake up thinking its friday 🙄 https://t.co/bKImYhaDAr',
       "One of the most dynamic players in the game.  Breaking down @run__cmc's game through True View highlights 💪 (via… https://t.co/Uv8XZ9FHLb",
       'News: A new restaurant and wine bar is coming to the North Shore, opening in the space where a BBQ spot had been. https://t.co/oCSHHs4YxQ',
       ..., '“WHAT’S IN THE BOX?!” https://t.co/TyVZYmPz64',
       "Philadelphia NAACP Takes Issue With DA's Ruling Against Appeal For Convicted Cop Killer Mumia Abu-Jamal.… https://t.co/VydMJm2xjp",
       'Federal Conservative leader Andrew Scheer speaks at the Energy Relaunch conference in Calgary #cdnpoli #ableg https://t.co/0cqmfyf5Gy'],
      dtype=object)

In [13]:
hashtag_mask = np.all(np.array(hashtag_positions)!=np.array([-1,-1]),axis=1)
np.array(hashtag_positions)[hashtag_mask]
train_sentences[hashtag_mask]

array(['https://t.co/JZ65RNKdYZ       Saturdays are looking a bit different this Fall. You can now catch @ESPNCFB live on @Hulu.  #ad',
       'Learn to appreciate what you have before time makes you appreciate what you had. #gratitude',
       "I am completely in love with the new song @KeshaRose it's lovely and moving. #KeshaIsBack",
       ...,
       "Oh and let's make this extra special. Tweet us what song you'd want to see us cover. #E3CoverRequest",
       'Here is a Free service on Fiverr. Claim it now! https://t.co/rZBjtmK5wz #WUVIP',
       'NewsInSG: Get HR policy right, and the opportunities are endless: Alvin Yapp: The BusAds Pte Ltd d... https://t.co/g5b8Nu2vIL #Singapore'],
      dtype=object)

In [14]:
user_mask = np.all(np.array(user_positions)!=np.array([-1,-1]),axis=1)
np.array(user_positions)[user_mask]
train_sentences[user_mask]

array(['Happy birthday family 🙏🏼 @jumpman_9',
       'Manageable Freedom?: https://t.co/52ZKAna7kU via @YouTube',
       'I just got... SO EXCITED for @VidCon!!!!!', ...,
       '#tradewar may snip 5% off China GDP  https://t.co/IJmlRLWNld  @praveenasharma3',
       'Want to win a $100 Amazon GC or PayPal Cash? https://t.co/bmE47mvATE via @DonnaChaffins',
       'Do You Feel Like You Live in a Battlefield? https://t.co/XytH4nGTS4 via @biblegateway'],
      dtype=object)

In [15]:
url_positions_test = []
hashtag_positions_test = []
user_positions_test = []
for test_sentence in test_sentences:
    test_sentence_split = test_sentence.split()
    urls = [i for i in test_sentence_split if (i.startswith('https:') or i.startswith('http:'))]
    if(len(urls)==1 and urls[0]==test_sentence_split[-1]):
        start = test_sentence.find(urls[0])
        end = start+len(urls[0])
        url_positions_test.append([start,end])
    else:
        url_positions_test.append([-1,-1])

    hashtags = [i for i in test_sentence_split if i.startswith('#')]
    if(len(hashtags)==1 and hashtags[0]==test_sentence_split[-1]):
        start = test_sentence.find(hashtags[0])
        end = start+len(hashtags[0])
        hashtag_positions_test.append([start,end])
    else:
        hashtag_positions_test.append([-1,-1])

    users = [i for i in test_sentence_split if i.startswith('@')]
    if(len(users)==1 and users[0]==test_sentence_split[-1]):
        start = test_sentence.find(users[0])
        end = start+len(users[0])
        user_positions_test.append([start,end])
    else:
        user_positions_test.append([-1,-1])

In [16]:
url_mask_test = np.all(np.array(url_positions_test)!=np.array([-1,-1]),axis=1)
np.array(url_positions_test)[url_mask_test]
test_sentences[url_mask_test]

array(['Now Playing: ♬ Dick Curless - Evil Hearted Me ♬ https://t.co/fzgP9IRt2h',
       'Not only are you comfortably swaddled in security today, it’s ... More for Capricorn https://t.co/MVCHEli4g1',
       'Do These Two Lines Match Up On Your Hands Here s What It Means https://t.co/pC5IFtVBxk',
       ..., 'Lol hey, boo. 🤗🤗 https://t.co/cmQ0125gXM',
       'When you get a fall wedding invitation in the mail: https://t.co/fwdBnOM42e',
       'CONFIRMED: Today is it for the season for one of our favorite places for black raspberry (or any) ice cream.… https://t.co/tkpt0oSn2I'],
      dtype=object)

In [17]:
hashtag_mask_test = np.all(np.array(hashtag_positions_test)!=np.array([-1,-1]),axis=1)
np.array(hashtag_positions_test)[hashtag_mask_test]
test_sentences[hashtag_mask_test]

array(["New Indie Book Release: THE UNNAMED GIRL (by Mike H Mizrahi @MikeHMiz) &gt;\xa0https://t.co/DVZgymret4 &lt; 'Power of Love' Historical #MustRead",
       'Man shot to death by EPS had rifle in stolen vehicle: ASIRT https://t.co/cZSaiP8V6i #yeg',
       'How to Delete Specific Safari History on Mac https://t.co/Jhz9QeAydF #AppleTips',
       ...,
       "Barack Obama's final words got me in tears 😭😭 https://t.co/njYcpJ6PxW #ObamaFarewell",
       'Who wants to make billions of dollars? AND . . . is a biological engineer? Please create palm trees that can live in Canada #DeciduousTrees',
       'Bangladesh: Landslides Threaten Rohingya Shelters - Human Rights Watch https://t.co/Kj7g24wxOu #Bangladesh'],
      dtype=object)

In [18]:
user_mask_test = np.all(np.array(user_positions_test)!=np.array([-1,-1]),axis=1)
np.array(user_positions_test)[user_mask_test]
test_sentences[user_mask_test]

array(['These strawberry sandwich cookies are so easy to make and so tasty! Perfect for #MothersDay https://t.co/Uq7cooR2y7 via @iamthemaven',
       'http://t.co/fVhH7zdpWU      WALMART Underground Tunnels &amp; INSIDER INFO PROMPT CLOSINGS https://t.co/s5BfH7Wlva via @YouTube',
       'LETS GOOOO!!!! Whole HEAP of world premiers on the show tonight. WHERE U AT??? &gt;&gt;&gt;&gt;&gt; @1xtra',
       ...,
       'People adopt made-up social rules to be part of a group https://t.co/om4gsHvVhD by @j_timmer',
       'The Power of Grace in Daily Life https://t.co/uwsSjcbc9r via @bay_art',
       'Writing About Family: Advice, Second Thoughts, Xanax, and a Note to My Mom https://t.co/QMm7pm1nHM @glimmertrain'],
      dtype=object)

In [19]:
url_positions_start = np.array(url_positions)[url_mask][:,0]
url_positions_end = np.array(url_positions)[url_mask][:,1]

hashtag_positions_start = np.array(hashtag_positions)[hashtag_mask][:,0]
hashtag_positions_end = np.array(hashtag_positions)[hashtag_mask][:,1]

user_positions_start = np.array(user_positions)[user_mask][:,0]
user_positions_end = np.array(user_positions)[user_mask][:,1]

In [20]:
url_positions_start_test = np.array(url_positions_test)[url_mask_test][:,0]
url_positions_end_test = np.array(url_positions_test)[url_mask_test][:,1]

hashtag_positions_start_test = np.array(hashtag_positions_test)[hashtag_mask_test][:,0]
hashtag_positions_end_test = np.array(hashtag_positions_test)[hashtag_mask_test][:,1]

user_positions_start_test = np.array(user_positions_test)[user_mask_test][:,0]
user_positions_end_test = np.array(user_positions_test)[user_mask_test][:,1]

In [21]:
train_sentences_url = ["_start_ " + "url?" + " _delimiter_ " + sentence + " _classify_" for sentence in train_sentences[url_mask]]
train_sentences_hashtag = ["_start_ " + "hashtag?" + " _delimiter_ " + sentence + " _classify_" for sentence in train_sentences[hashtag_mask]]
train_sentences_user = ["_start_ " + "user?" + " _delimiter_ " + sentence + " _classify_" for sentence in train_sentences[user_mask]]

In [22]:
test_sentences_url = ["_start_ " + "url?" + " _delimiter_ " + sentence + " _classify_" for sentence in test_sentences[url_mask_test]]
test_sentences_hashtag = ["_start_ " + "hashtag?" + " _delimiter_ " + sentence + " _classify_" for sentence in test_sentences[hashtag_mask_test]]
test_sentences_user = ["_start_ " + "user?" + " _delimiter_ " + sentence + " _classify_" for sentence in test_sentences[user_mask_test]]

In [23]:
MAX_LENGTH = 128

In [24]:
train_sentences_encoded = [tokenizer.encode(sent + " _classify_", add_prefix_space=True) for sent in train_sentences[url_mask]] 
prefix = tokenizer.encode("_start_ " + "url?" + " _delimiter_ " )
input_ids_url = [prefix + encoded_sent  for encoded_sent in train_sentences_encoded]
len_a =len(prefix)
len_b = MAX_LENGTH-len(prefix)
token_type_ids_url = [[0 for x in range(0,len_a)]+[1 for x in range(0,len_b)] for i in range(0,len(train_sentences[url_mask]))]

train_sentences_encoded = [tokenizer.encode(sent + " _classify_", add_prefix_space=True) for sent in train_sentences[hashtag_mask]] 
prefix = tokenizer.encode("_start_ " + "hashtag?" + " _delimiter_ ")
input_ids_hashtag = [prefix + encoded_sent for encoded_sent in train_sentences_encoded]
len_a =len(prefix)
len_b = MAX_LENGTH-len(prefix)
token_type_ids_hashtag = [[0 for x in range(0,len_a)]+[1 for x in range(0,len_b)] for i in range(0,len(train_sentences[hashtag_mask]))]

train_sentences_encoded = [tokenizer.encode(sent + " _classify_", add_prefix_space=True) for sent in train_sentences[user_mask]]
prefix = tokenizer.encode("_start_ " + "user?" + " _delimiter_ ")
input_ids_user = [prefix + encoded_sent for encoded_sent in train_sentences_encoded]
len_a =len(prefix)
len_b = MAX_LENGTH-len(prefix)
token_type_ids_user = [[0 for x in range(0,len_a)]+[1 for x in range(0,len_b)] for i in range(0,len(train_sentences[user_mask]))]

In [26]:
test_sentences_encoded = [tokenizer.encode(sent + " _classify_", add_prefix_space=True) for sent in test_sentences[url_mask_test]] 
prefix = tokenizer.encode("_start_ " + "url?" + " _delimiter_ " )
input_ids_url_test = [prefix + encoded_sent  for encoded_sent in test_sentences_encoded]
len_a =len(prefix)
len_b = MAX_LENGTH-len(prefix)
token_type_ids_url_test = [[0 for x in range(0,len_a)]+[1 for x in range(0,len_b)] for i in range(0,len(test_sentences[url_mask_test]))]

test_sentences_encoded = [tokenizer.encode(sent + " _classify_", add_prefix_space=True) for sent in test_sentences[hashtag_mask_test]] 
prefix = tokenizer.encode("_start_ " + "hashtag?" + " _delimiter_ ")
input_ids_hashtag_test = [prefix + encoded_sent for encoded_sent in test_sentences_encoded]
len_a =len(prefix)
len_b = MAX_LENGTH-len(prefix)
token_type_ids_hashtag_test = [[0 for x in range(0,len_a)]+[1 for x in range(0,len_b)] for i in range(0,len(test_sentences[hashtag_mask_test]))]

test_sentences_encoded = [tokenizer.encode(sent + " _classify_", add_prefix_space=True) for sent in test_sentences[user_mask_test]]
prefix = tokenizer.encode("_start_ " + "user?" + " _delimiter_ ")
input_ids_user_test = [prefix + encoded_sent for encoded_sent in test_sentences_encoded]
len_a =len(prefix)
len_b = MAX_LENGTH-len(prefix)
token_type_ids_user_test = [[0 for x in range(0,len_a)]+[1 for x in range(0,len_b)] for i in range(0,len(test_sentences[user_mask_test]))]

In [27]:
def is_Sublist(l, s):
    i=-1
    sub_set = False
    if s == []:
        sub_set = True
    elif s == l:
        sub_set = True
    elif len(s) > len(l):
        sub_set = False

    else:
        for i in range(len(l)):
            if l[i] == s[0]:
                n = 1
                while (n < len(s)) and (l[i+n] == s[n]):
                    n += 1

                if n == len(s):
                    sub_set = True
                    break

    return sub_set,i

start and end positions are relative to the original sentence (train_sentences_url), we need to convert it to the position in input_ids_url

In [28]:
url_positions_start_ids = []
url_positions_end_ids = []
offset = len("_start_ " + "url?" + " _delimiter_ ")
for i in range(0,len(train_sentences_url)):
    if(url_positions_start[i]!=-1 and url_positions_end[i]!=-1):
        url_ids = tokenizer.encode(train_sentences_url[i][offset+url_positions_start[i]:offset+url_positions_end[i]], add_prefix_space=True )
        result = is_Sublist(input_ids_url[i],url_ids)
        start = result[1]
        end = start+len(url_ids)
        url_positions_start_ids.append(start)
        url_positions_end_ids.append(end)
    else:
        url_positions_start_ids.append(-1)
        url_positions_end_ids.append(-1)

In [29]:
url_positions_start_ids_test = []
url_positions_end_ids_test = []
offset = len("_start_ " + "url?" + " _delimiter_ ")
for i in range(0,len(test_sentences_url)):
    if(url_positions_start_test[i]!=-1 and url_positions_end_test[i]!=-1):
        url_ids = tokenizer.encode(test_sentences_url[i][offset+url_positions_start_test[i]:offset+url_positions_end_test[i]], add_prefix_space=True)
        result = is_Sublist(input_ids_url_test[i],url_ids)
        start = result[1]
        end = start+len(url_ids)
        url_positions_start_ids_test.append(start)
        url_positions_end_ids_test.append(end)
    else:
        url_positions_start_ids_test.append(-1)
        url_positions_end_ids_test.append(-1)

In [30]:
hashtag_positions_start_ids = []
hashtag_positions_end_ids = []
offset = len("_start_ " + "hashtag?" + " _delimiter_ ")
for i in range(0,len(train_sentences_hashtag)):
    if(hashtag_positions_start[i]!=-1 and hashtag_positions_end[i]!=-1):
        hashtag_ids = tokenizer.encode(train_sentences_hashtag[i][offset+hashtag_positions_start[i]:offset+hashtag_positions_end[i]], add_prefix_space=True)
        result = is_Sublist(input_ids_hashtag[i],hashtag_ids)
        start = result[1]
        end = start+len(hashtag_ids)
        hashtag_positions_start_ids.append(start)
        hashtag_positions_end_ids.append(end)
    else:
        hashtag_positions_start_ids.append(-1)
        hashtag_positions_end_ids.append(-1)

In [31]:
hashtag_positions_start_ids_test = []
hashtag_positions_end_ids_test = []
offset = len("_start_ " + "hashtag?" + " _delimiter_ ")
for i in range(0,len(test_sentences_hashtag)):
    if(hashtag_positions_start_test[i]!=-1 and hashtag_positions_end_test[i]!=-1):
        hashtag_ids = tokenizer.encode(test_sentences_hashtag[i][offset+hashtag_positions_start_test[i]:offset+hashtag_positions_end_test[i]], add_prefix_space=True)
        result = is_Sublist(input_ids_hashtag_test[i],hashtag_ids)
        start = result[1]
        end = start+len(hashtag_ids)
        hashtag_positions_start_ids_test.append(start)
        hashtag_positions_end_ids_test.append(end)
    else:
        hashtag_positions_start_ids_test.append(-1)
        hashtag_positions_end_ids_test.append(-1)

In [32]:
user_positions_start_ids = []
user_positions_end_ids = []
offset = len("_start_ " + "user?" + " _delimiter_ ")
for i in range(0,len(train_sentences_user)):
    if(user_positions_start[i]!=-1 and user_positions_end[i]!=-1):
        user_ids = tokenizer.encode(train_sentences_user[i][offset+user_positions_start[i]:offset+user_positions_end[i]], add_prefix_space=True)
        result = is_Sublist(input_ids_user[i],user_ids)
        start = result[1]
        end = start+len(user_ids)
        user_positions_start_ids.append(start)
        user_positions_end_ids.append(end)
    else:
        user_positions_start_ids.append(-1)
        user_positions_end_ids.append(-1)

In [33]:
user_positions_start_ids_test = []
user_positions_end_ids_test = []
offset = len("_start_ " + "user?" + " _delimiter_ ")
for i in range(0,len(test_sentences_user)):
    if(user_positions_start_test[i]!=-1 and user_positions_end_test[i]!=-1):
        user_ids = tokenizer.encode(test_sentences_user[i][offset+user_positions_start_test[i]:offset+user_positions_end_test[i]], add_prefix_space=True)
        result = is_Sublist(input_ids_user_test[i],user_ids)
        start = result[1]
        end = start+len(user_ids)
        user_positions_start_ids_test.append(start)
        user_positions_end_ids_test.append(end)
    else:
        user_positions_start_ids_test.append(-1)
        user_positions_end_ids_test.append(-1)

In [34]:
MAX_LEN = 128

In [35]:
train_input_ids_url = pad_sequences(input_ids_url, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
train_input_ids_hashtag = pad_sequences(input_ids_hashtag, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
train_input_ids_user = pad_sequences(input_ids_user, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [36]:
test_input_ids_url = pad_sequences(input_ids_url_test, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_input_ids_hashtag = pad_sequences(input_ids_hashtag_test, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_input_ids_user = pad_sequences(input_ids_user_test, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [37]:
train_inputs_url = torch.tensor(train_input_ids_url)
train_inputs_hashtag = torch.tensor(train_input_ids_hashtag)
train_inputs_user = torch.tensor(train_input_ids_user)

train_url_positions_start = torch.tensor(url_positions_start_ids)
train_hashtag_positions_start = torch.tensor(hashtag_positions_start_ids)
train_user_positions_start = torch.tensor(user_positions_start_ids)

train_url_positions_end = torch.tensor(url_positions_end_ids)
train_hashtag_positions_end = torch.tensor(hashtag_positions_end_ids)
train_user_positions_end = torch.tensor(user_positions_end_ids)

train_token_type_url = torch.tensor(token_type_ids_url)
train_token_type_hashtag = torch.tensor(token_type_ids_hashtag)
train_token_type_user = torch.tensor(token_type_ids_user)

In [38]:
test_inputs_url = torch.tensor(test_input_ids_url)
test_inputs_hashtag = torch.tensor(test_input_ids_hashtag)
test_inputs_user = torch.tensor(test_input_ids_user)

test_url_positions_start = torch.tensor(url_positions_start_ids_test)
test_hashtag_positions_start = torch.tensor(hashtag_positions_start_ids_test)
test_user_positions_start = torch.tensor(user_positions_start_ids_test)

test_url_positions_end = torch.tensor(url_positions_end_ids_test)
test_hashtag_positions_end = torch.tensor(hashtag_positions_end_ids_test)
test_user_positions_end = torch.tensor(user_positions_end_ids_test)

test_token_type_url = torch.tensor(token_type_ids_url_test)
test_token_type_hashtag = torch.tensor(token_type_ids_hashtag_test)
test_token_type_user = torch.tensor(token_type_ids_user_test)

In [39]:
train_inputs = torch.cat((train_inputs_url,train_inputs_hashtag, train_inputs_user),0)
train_positions_start = torch.cat((train_url_positions_start,train_hashtag_positions_start, train_user_positions_start),0)
train_positions_end = torch.cat((train_url_positions_end,train_hashtag_positions_end, train_user_positions_end),0)
train_token_type = torch.cat((train_token_type_url,train_token_type_hashtag, train_token_type_user),0)

In [40]:
test_inputs = torch.cat((test_inputs_url,test_inputs_hashtag, test_inputs_user),0)
test_positions_start = torch.cat((test_url_positions_start,test_hashtag_positions_start, test_user_positions_start),0)
test_positions_end = torch.cat((test_url_positions_end,test_hashtag_positions_end, test_user_positions_end),0)
test_token_type = torch.cat((test_token_type_url,test_token_type_hashtag, test_token_type_user),0)

### Create the generators

In [41]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
batch_size = 8

In [42]:
train_data = TensorDataset(train_inputs, 
                           train_positions_start, 
                           train_positions_end,
                           train_token_type)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [43]:
test_data = TensorDataset(test_inputs, 
                           test_positions_start, 
                           test_positions_end, 
                           test_token_type)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [44]:
linear = nn.Linear(768, 2).to(torch.device("cuda:0"))

In [45]:
param_optimizer = list(linear.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5,)

## Training

In [46]:
model = model.transformer

In [107]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  linear.train()
    
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    ##batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_inputs, b_positions_start, b_positions_end, b_token_type = batch
        
    sequence_output = model(b_inputs.to(device), token_type_ids=b_token_type.to(device))[0]
    logits = linear(sequence_output)
    start_logits, end_logits = logits.split(1, dim=-1)
    start_logits = start_logits.squeeze(-1)
    end_logits = end_logits.squeeze(-1)
    start_positions = b_positions_start.to(device)
    end_positions = b_positions_end.to(device)
    if len(start_positions.size()) > 1:
        start_positions = start_positions.squeeze(-1)
    if len(end_positions.size()) > 1:
        end_positions = end_positions.squeeze(-1)
    ignored_index = start_logits.size(1)
    start_positions.clamp_(0, ignored_index)
    end_positions.clamp_(0, ignored_index)
    loss_fct = nn.CrossEntropyLoss(ignore_index=ignored_index)
    start_loss = loss_fct(start_logits, start_positions )
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss) / 2
    total_loss.backward(retain_graph=True)
    optimizer.step()

Epoch: 100%|██████████| 2/2 [1:59:59<00:00, 3599.78s/it]  


## Evaluation

In [32]:
test_data_url = TensorDataset(test_inputs_url, 
                           test_token_type_url)
test_sampler_url = torch.utils.data.SequentialSampler(test_data_url)
test_dataloader_url = DataLoader(test_data_url, sampler=test_sampler_url, batch_size=batch_size)

In [33]:
linear.eval()

preds_url = []

for batch in test_dataloader_url:
    b_inputs, b_token_type = batch
    with torch.no_grad():
        sequence_output = model(b_inputs.to(device), token_type_ids=b_token_type.to(device))[0]
        logits = linear(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
    
    start_logits = start_logits.detach().cpu().numpy()
    end_logits = end_logits.detach().cpu().numpy()
    preds_url.append([start_logits,end_logits])

In [34]:
test_data_hashtag = TensorDataset(test_inputs_hashtag, 
                           test_token_type_hashtag)
test_sampler_hashtag = torch.utils.data.SequentialSampler(test_data_hashtag)
test_dataloader_hashtag = DataLoader(test_data_hashtag, sampler=test_sampler_hashtag, batch_size=batch_size)

In [35]:
preds_hashtag = []

for batch in test_dataloader_hashtag:
    b_inputs, b_token_type = batch
    with torch.no_grad():
        sequence_output = model(b_inputs.to(device), token_type_ids=b_token_type.to(device))[0]
        logits = linear(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
    
    start_logits = start_logits.detach().cpu().numpy()
    end_logits = end_logits.detach().cpu().numpy()
    preds_hashtag.append([start_logits,end_logits])

In [36]:
test_data_user = TensorDataset(test_inputs_user, 
                           test_token_type_user)
test_sampler_user = torch.utils.data.SequentialSampler(test_data_user)
test_dataloader_user = DataLoader(test_data_user, sampler=test_sampler_user, batch_size=batch_size)

In [37]:
preds_user = []

for batch in test_dataloader_user:
    b_inputs, b_token_type = batch
    with torch.no_grad():
        sequence_output = model(b_inputs.to(device), token_type_ids=b_token_type.to(device))[0]
        logits = linear(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
    
    start_logits = start_logits.detach().cpu().numpy()
    end_logits = end_logits.detach().cpu().numpy()
    preds_user.append([start_logits,end_logits])

In [38]:
predictions_url_start = []
predictions_url_end = []
for preds in preds_url:
    predictions_url_start = predictions_url_start + list(np.argmax(preds[0],axis=1))
    predictions_url_end = predictions_url_end + list(np.argmax(preds[1],axis=1))

In [39]:
predictions_hashtag_start = []
predictions_hashtag_end = []
for preds in preds_hashtag:
    predictions_hashtag_start = predictions_hashtag_start + list(np.argmax(preds[0],axis=1))
    predictions_hashtag_end = predictions_hashtag_end + list(np.argmax(preds[1],axis=1))

In [40]:
predictions_user_start = []
predictions_user_end = []
for preds in preds_user:
    predictions_user_start = predictions_user_start + list(np.argmax(preds[0],axis=1))
    predictions_user_end = predictions_user_end + list(np.argmax(preds[1],axis=1))

In [122]:
print(classification_report([0 if pos<0 else pos for pos in url_positions_start_ids_test], predictions_url_start, digits = 4))
print(classification_report([0 if pos<0 else pos for pos in url_positions_end_ids_test], predictions_url_end, digits = 4))

             precision    recall  f1-score   support

          1     0.0000    0.0000    0.0000         0
          4     0.0000    0.0000    0.0000         0
          5     0.9518    0.9937    0.9723       159
          6     0.9834    1.0000    0.9916       414
          7     0.9908    1.0000    0.9954       643
          8     0.9933    0.9967    0.9950       899
          9     0.9889    0.9925    0.9907       805
         10     0.9769    1.0000    0.9883       887
         11     0.9859    0.9934    0.9896      1058
         12     0.9784    0.9904    0.9843      1143
         13     0.9716    0.9874    0.9794      1110
         14     0.9869    0.9951    0.9910      1214
         15     0.9839    0.9871    0.9855      1242
         16     0.9813    0.9798    0.9806      1235
         17     0.9816    0.9840    0.9828      1190
         18     0.9785    0.9880    0.9832      1245
         19     0.9848    0.9870    0.9859      1312
         20     0.9814    0.9837    0.9826   

/home/test/anaconda3/envs/jupyter/lib/python3.6/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/test/anaconda3/envs/jupyter/lib/python3.6/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [123]:
print(classification_report([0 if pos<0 else pos for pos in hashtag_positions_start_ids_test], predictions_hashtag_start, digits = 4))
print(classification_report([0 if pos<0 else pos for pos in hashtag_positions_end_ids_test], predictions_hashtag_end, digits = 4))

             precision    recall  f1-score   support

          6     0.6667    1.0000    0.8000         2
          7     0.9167    0.7857    0.8462        14
          8     0.9444    0.8095    0.8718        21
          9     0.7297    0.9000    0.8060        30
         10     0.9459    0.8537    0.8974        41
         11     0.8293    0.9714    0.8947        35
         12     0.7692    0.9375    0.8451        32
         13     0.9242    0.9839    0.9531        62
         14     0.7647    0.9286    0.8387        56
         15     0.8333    0.9286    0.8784        70
         16     0.8539    0.9383    0.8941        81
         17     0.8556    0.9390    0.8953        82
         18     0.8652    0.9390    0.9006        82
         19     0.8140    0.9459    0.8750        74
         20     0.8675    0.9600    0.9114        75
         21     0.8406    0.9831    0.9062        59
         22     0.7895    0.8824    0.8333        68
         23     0.8632    0.9425    0.9011   

/home/test/anaconda3/envs/jupyter/lib/python3.6/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [124]:
print(classification_report([0 if pos<0 else pos for pos in user_positions_start_ids_test], predictions_user_start, digits = 4))
print(classification_report([0 if pos<0 else pos for pos in user_positions_end_ids_test], predictions_user_end, digits = 4))

             precision    recall  f1-score   support

          5     1.0000    1.0000    1.0000         1
          6     0.8571    1.0000    0.9231         6
          7     0.7308    0.8261    0.7755        23
          8     0.7778    0.8750    0.8235         8
          9     0.9412    1.0000    0.9697        16
         10     0.7778    0.9333    0.8485        15
         11     0.7500    1.0000    0.8571        15
         12     0.5789    1.0000    0.7333        11
         13     0.4643    0.9286    0.6190        14
         14     0.3182    0.8750    0.4667         8
         15     0.5455    0.8571    0.6667        14
         16     0.4211    0.8000    0.5517        10
         17     0.5556    0.7692    0.6452        13
         18     0.3913    0.8182    0.5294        11
         19     0.5862    1.0000    0.7391        17
         20     0.5625    0.8182    0.6667        22
         21     0.4348    0.7692    0.5556        13
         22     0.4815    0.7647    0.5909   

/home/test/anaconda3/envs/jupyter/lib/python3.6/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
print(classification_report([('0' if pos_start<0 else str(pos_start))+","+('0' if pos_end<0 else str(pos_end)) for pos_start,pos_end in zip(url_positions_start_ids_test,url_positions_end_ids_test)], [str(pos_start)+","+str(pos_end) for pos_start,pos_end in zip(predictions_url_start,predictions_url_end)], digits = 4))
print(classification_report([('0' if pos_start<0 else str(pos_start))+","+('0' if pos_end<0 else str(pos_end)) for pos_start,pos_end in zip(hashtag_positions_start_ids_test,hashtag_positions_end_ids_test)], [str(pos_start)+","+str(pos_end) for pos_start,pos_end in zip(predictions_hashtag_start,predictions_hashtag_end)], digits = 4))
print(classification_report([('0' if pos_start<0 else str(pos_start))+","+('0' if pos_end<0 else str(pos_end)) for pos_start,pos_end in zip(user_positions_start_ids_test,user_positions_end_ids_test)], [str(pos_start)+","+str(pos_end) for pos_start,pos_end in zip(predictions_user_start,predictions_user_end)], digits = 4))

             precision    recall  f1-score   support

       1,18     0.0000    0.0000    0.0000         0
      10,21     1.0000    1.0000    1.0000         4
      10,22     0.9872    1.0000    0.9935        77
      10,23     0.9926    1.0000    0.9963       268
      10,24     1.0000    1.0000    1.0000       325
      10,25     0.9944    1.0000    0.9972       178
      10,26     0.9189    1.0000    0.9577        34
      10,27     0.0000    0.0000    0.0000         0
      10,28     0.0000    0.0000    0.0000         0
      10,29     0.0000    0.0000    0.0000         0
      10,30     0.0000    0.0000    0.0000         1
      10,31     0.0000    0.0000    0.0000         0
      10,34     0.0000    0.0000    0.0000         0
      10,35     0.0000    0.0000    0.0000         0
      10,39     0.0000    0.0000    0.0000         0
      10,41     0.0000    0.0000    0.0000         0
    100,114     1.0000    1.0000    1.0000         1
    104,117     1.0000    1.0000    1.0000   

/home/test/anaconda3/envs/jupyter/lib/python3.6/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/test/anaconda3/envs/jupyter/lib/python3.6/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
